In [10]:
from pathlib import Path
import torch
import tqdm

from au2v.config import ModelConfig, TrainerConfig
from au2v.dataset_manager import load_dataset_manager
from au2v.trainer import PyTorchTrainer, calc_accuracy
from au2v.model import load_model, PyTorchModel

In [ ]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    hit_counts = {k: 0 for k in top_k}
    for seq_index, context_items, target_indices in tqdm.tqdm(test_dataset):
        rec_list = model.output_rec_lists(
            seq_index=torch.LongTensor([seq_index]),
            item_indices=torch.LongTensor([context_items]),
            cand_item_indices=torch.arange(num_item),
            k=max(top_k),
        )
        for k in top_k:
            hit_counts[k] += len(set(target_indices) & set(rec_list[0][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [12]:
models = ["old-attentive", "doc2vec", "attentive"]

for model_name in models:
    model_config = ModelConfig(
        weight_decay=1e-8,
        max_embedding_norm=1.0,
        d_model=64,
        lr=1e-4 if model_name == "doc2vec" else 5e-5
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        model_name=model_name,
        load_dataset=False,
        save_dataset=False,
        load_model=False,
        ignore_saved_model=True,
        epochs=5,
    )

    print(model_config)
    print(trainer_config)

    dataset_manager = load_dataset_manager(
        dataset_name=trainer_config.dataset_name,
        dataset_dir=trainer_config.dataset_dir,
        data_dir="../data/",
        load_dataset=trainer_config.load_dataset,
        save_dataset=trainer_config.save_dataset,
        window_size=model_config.window_size,
    )
    model = load_model(
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer.fit()

ModelConfig(d_model=64, init_embedding_std=0.2, max_embedding_norm=1.0, window_size=5, negative_sample_size=5, lr=5e-05, weight_decay=1e-08)
TrainerConfig(model_name='old-attentive', dataset_name='movielens', epochs=5, batch_size=64, verbose=False, ignore_saved_model=True, load_model=False, save_model=True, load_dataset=False, save_dataset=False, model_dir='cache/model/', dataset_dir='cache/dataset/', device='cpu')
dataset_manager does not exist at: cache/dataset/movielens.pickle, create dataset
num_seq: 6040, num_item: 3706, num_item_meta: 28, num_seq_meta: 30, num_item_meta_types: 3, num_seq_meta_types: 3
to_sequential_data start
to_sequential_data end


100%|██████████| 9190/9190 [02:02<00:00, 75.06it/s]


train 0.006227718123920886


100%|██████████| 2298/2298 [00:09<00:00, 244.99it/s]


valid 0.005795715705987552
saved model to cache/model/movielens/old-attentive.pt


...

train 0.005122879080087255


100%|██████████| 2298/2298 [00:12<00:00, 190.35it/s]


valid 0.005124912388533432
saved model to cache/model/movielens/attentive.pt


100%|██████████| 1591/1591 [00:18<00:00, 85.23it/s]

{'Accuracy@10': 0.1744186046511628, 'Accuracy@30': 0.18013827781269642, 'Accuracy@50': 0.18154619736015085}


In [13]:
models = ["attentive", "doc2vec", "old-attentive"]

for model_name in models:
    model = torch.load(f"cache/model/movielens/{model_name}.pt")

    print(
        model_name,
        calc_accuracy(
            model=model,
            num_item=dataset_manager.num_item,
            test_dataset=dataset_manager.test_datasets["test"],
            top_k=[10, 30, 50],
        ),
    )

100%|██████████| 1591/1591 [00:18<00:00, 86.71it/s]


attentive {'Accuracy@10': 0.1744186046511628, 'Accuracy@30': 0.18013827781269642, 'Accuracy@50': 0.18154619736015085}


100%|██████████| 1591/1591 [00:08<00:00, 191.86it/s]


doc2vec {'Accuracy@10': 0.17165304839723444, 'Accuracy@30': 0.16647810601298973, 'Accuracy@50': 0.15751099937146448}


100%|██████████| 1591/1591 [00:18<00:00, 84.60it/s]

old-attentive {'Accuracy@10': 0.177561282212445, 'Accuracy@30': 0.16614288707311964, 'Accuracy@50': 0.15803896920175992}
